In [1]:
import ROOT as root
import time

Welcome to JupyROOT 6.28/04


In [2]:
chain = root.TChain('tree')
chain.Add('./bkg_tree2_ZZTo2L2Nu_s.root')

1

In [3]:
print(type(chain))
n = chain.GetEntries()
print("\nTotal no. of events : ",chain.GetEntries())

<class cppyy.gbl.TChain at 0x557d4a020840>

Total no. of events :  1071295


In [4]:
import ROOT

def print_leaves(tree):
    for branch in tree.GetListOfBranches():
        print(branch.GetName())

# Open the ROOT file
file = ROOT.TFile("bkg_tree2_ZZTo2L2Nu_s.root")

# Access the tree or chain
tree = file.Get("tree")

# Print the leaves
print_leaves(tree)

# Close the file
file.Close()

dR
dPhiMET_u1
mass_uu
MET_pt
pTu1
pTu2
Nele
Njet
Nbjet
Nmuon
HpT_jet


In [5]:
cutflow_H  = root.TH1D('cutflow','Cut Flow Histogram', 7, -0.5, 6.5)

In [6]:
cutflow_H.Reset()

In [7]:
def RootfileWrite():
    f2 = root.TFile("CutFlow_bkg_ZZTo2L2Nu_s.root", "RECREATE")
    cutflow_H.Write()
    f2.Write()
    f2.Close()

In [8]:
start = time.time()    # note the starting time

for i,ent in enumerate(chain):
    
#     if i>1000000: break;
    if (i > 0 and i%100000 == 0): print("Processing: ", i, "th event!")
    
    cutflow_H.Fill(0)
    
    if ent.mass_uu < 70 or ent.mass_uu > 110:continue
    cutflow_H.Fill(1)
    
    if ent.MET_pt  > 40:continue
    cutflow_H.Fill(2)
    
    if ent.Nele    > 0:continue
    cutflow_H.Fill(3)
    
    if ent.Nbjet    > 0:continue
    cutflow_H.Fill(4)
    
    if ent.Njet   > 3:continue
    cutflow_H.Fill(5)
    
    if ent.Njet < 3 and ent.HpT_jet >60:continue
    cutflow_H.Fill(6)

print ("Loop is completed")     # LOOP COMPLETTION MASSAGE
end = time.time()   # Note the end time
print(end-start)    # Show the time taken in seconds to rum the code
RootfileWrite()

Processing:  100000 th event!
Processing:  200000 th event!
Processing:  300000 th event!
Processing:  400000 th event!
Processing:  500000 th event!
Processing:  600000 th event!
Processing:  700000 th event!
Processing:  800000 th event!
Processing:  900000 th event!
Processing:  1000000 th event!
Loop is completed
10.979434967041016


In [9]:
# Draw the histogram
%jsroot on
root.gStyle.SetOptStat(1)
root.gStyle.SetTextFont(42)

c1 = root.TCanvas('c1', 'c1', 1000, 600)
c1.SetLogy()
c1.SetGrid()

cutflow_H.GetXaxis().SetTitle("Number of cuts")
cutflow_H.GetYaxis().SetTitle("Event Number")

cutflow_H.GetXaxis().SetTitleSize(0.04)
cutflow_H.GetYaxis().SetTitleSize(0.04)
cutflow_H.Draw()

c1.Draw()

# Print histogram statistics
hist_name = cutflow_H.GetName()
entries = cutflow_H.GetEntries()
mean = cutflow_H.GetMean()
std_dev = cutflow_H.GetStdDev()
underflow = cutflow_H.GetBinContent(0)
overflow = cutflow_H.GetBinContent(cutflow_H.GetNbinsX() + 1)
integral = cutflow_H.Integral()
skewness = cutflow_H.GetSkewness()
kurtosis = cutflow_H.GetKurtosis()

print("Histogram Name:", hist_name)
print("Entries:", entries)
print("Mean:", mean)
print("Std Dev:", std_dev)
print("Underflow:", underflow)
print("Overflow:", overflow)
print("Integral:", integral)
print("Skewness:", skewness)
print("Kurtosis:", kurtosis)


Histogram Name: cutflow
Entries: 3407636.0
Mean: 1.8463315917545184
Std Dev: 1.9513124665934476
Underflow: 0.0
Overflow: 0.0
Integral: 3407636.0
Skewness: 0.8729917078902293
Kurtosis: -0.5669649110706803


In [10]:
H = cutflow_H
print("Number of bins: ",H.GetNbinsX())
x_low = H.GetXaxis().GetXmin()
x_high = H.GetXaxis().GetXmax()

bin_indices    = [] 
bin_contents   = []
bin_edges_low  = []
bin_edges_high = []
cut_effi_arr   = []
Ndenominator   = H.GetBinContent(1)

print("\t","Bin Content\tBin Error\tBin centers\tBin widths\tEdge_low\tEdge_high\tCut_Effi(%)")
for bin in range(1, H.GetNbinsX() + 1):
    bin_indices.append(bin)
    bin_content = H.GetBinContent(bin)
    cut_effi = round((bin_content*100/Ndenominator),3)
    cut_effi_arr.append(cut_effi)
    bin_contents.append(bin_content)
    bin_error = round(H.GetBinError(bin),3)
    bin_centers = round(H.GetXaxis().GetBinCenter(bin),3)
    bin_widths = H.GetXaxis().GetBinWidth(bin)
    edge_low = round(H.GetXaxis().GetBinLowEdge(bin),3)
    bin_edges_low.append(edge_low)
    edge_up = round(H.GetXaxis().GetBinLowEdge(bin+1),3)
    bin_edges_high.append(edge_up)
    
    print("Bin ",bin,": ",bin_content,"\t",bin_error,"\t",bin_centers,"\t\t",bin_widths,"\t\t",edge_low,"\t\t",edge_up,"\t\t",cut_effi)

Number of bins:  7
	 Bin Content	Bin Error	Bin centers	Bin widths	Edge_low	Edge_high	Cut_Effi(%)
Bin  1 :  1071295.0 	 1035.034 	 0.0 		 1.0 		 -0.5 		 0.5 		 100.0
Bin  2 :  1000268.0 	 1000.134 	 1.0 		 1.0 		 0.5 		 1.5 		 93.37
Bin  3 :  274016.0 	 523.465 	 2.0 		 1.0 		 1.5 		 2.5 		 25.578
Bin  4 :  273424.0 	 522.9 	 3.0 		 1.0 		 2.5 		 3.5 		 25.523
Bin  5 :  269830.0 	 519.452 	 4.0 		 1.0 		 3.5 		 4.5 		 25.187
Bin  6 :  269084.0 	 518.733 	 5.0 		 1.0 		 4.5 		 5.5 		 25.118
Bin  7 :  249719.0 	 499.719 	 6.0 		 1.0 		 5.5 		 6.5 		 23.31
